# Gallery Example: M/M/k Queue (Multi-Server)

This example demonstrates an M/M/k queueing system:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: k parallel servers (default k=3)
- **Capacity**: Infinite
- **Scheduling**: FCFS with multiple servers

Multi-server queues are fundamental models for systems with parallel service capacity.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mmk(k=3): Network {    """Create M/M/k queueing model        Parameters:    - k: Number of servers (default 3)    """    model = Network(f"M/M/{k}")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    queue.setNumberOfServers(k)  # Set number of servers    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    source.setArrival(oclass, Exp(2))     # λ = 2    queue.setService(oclass, Exp(1))      # μ = 1 per server, so total μ = k        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the model with default k=3 serversk_servers = 3model = gallery_mmk(k_servers)println(f"Servers: {k_servers}")println(f"Arrival rate: λ = 2")println(f"Service rate per server: μ = 1")println(f"Total service capacity: {k_servers} × 1 = {k_servers}")println(f"System utilization: ρ = λ/(k×μ) = 2/{k_servers} = {2/k_servers:.3f}")

## Theoretical Analysis for M/M/k

For M/M/k with λ=2, μ=1 per server, k=3:
- **Traffic Intensity**: a = λ/μ = 2/1 = 2
- **System Utilization**: ρ = a/k = 2/3 ≈ 0.667
- **Stability**: System is stable since ρ < 1

The M/M/k model is more complex than M/M/1 due to multiple servers:
- Jobs can be served immediately if any server is free
- Queueing only occurs when all k servers are busy
- Performance significantly better than equivalent M/M/1

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare M/M/k with equivalent M/M/1
println("\n=== Comparison with Equivalent M/M/1 ===")

fun create_equivalent_mm1(): Network {
    """Create M/M/1 with same total service capacity"""
    val model_mm1 = Network("M/M/1-Equivalent")
    val source = Source(model_mm1, "Source")
    val queue = Queue(model_mm1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_mm1, "Sink")
    
    val oclass = OpenClass(model_mm1, "Class")
    source.setArrival(oclass, Exp(2))     # Same arrival rate
    queue.setService(oclass, Exp(3))      # Total service rate = k×μ = 3
    
    val P = model_mm1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

val model_mm1 = create_equivalent_mm1()
val solver_mmk = MVA(model)
val solver_mm1 = MVA(model_mm1)

val avg_table_mmk = solver_mmk.avgTable
val avg_table_mm1 = solver_mm1.avgTable
// Extract performance metrics
val mmk_util = float(avg_table_mmk.iloc[1, 1])      # M/M/k utilization
val mmk_resp = float(avg_table_mmk.iloc[1, 2])      # M/M/k response time
val mmk_length = float(avg_table_mmk.iloc[1, 3])    # M/M/k queue length

val mm1_util = float(avg_table_mm1.iloc[1, 1])      # M/M/1 utilization
val mm1_resp = float(avg_table_mm1.iloc[1, 2])      # M/M/1 response time
val mm1_length = float(avg_table_mm1.iloc[1, 3])    # M/M/1 queue length

println(f"Performance Comparison (same total service capacity):")
println(f"")
println(f"M/M/{k_servers} (multi-server):")
println(f"  Utilization:   {mmk_util:.4f}")
println(f"  Response Time: {mmk_resp:.4f}")
println(f"  Queue Length:  {mmk_length:.4f}")
println(f"")
println(f"M/M/1 (single fast server):")
println(f"  Utilization:   {mm1_util:.4f}")
println(f"  Response Time: {mm1_resp:.4f}")
println(f"  Queue Length:  {mm1_length:.4f}")
println(f"")
println(f"Multi-server advantage:")
println(f"  Response time: {(mm1_resp / mmk_resp):.2f}x better with multiple servers")
println(f"  Queue length:  {(mm1_length / mmk_length):.2f}x lower with multiple servers")
println(f"")
println(f"Note: Multiple servers provide better performance than a single fast server")
println(f"due to reduced waiting time variability and improved resource utilization.")

In [ ]:
// Analyze scaling with number of servers
println("\n=== Server Scaling Analysis ===")

fun analyze_mmk_scaling(max_k=6): Network {
    """Analyze performance as number of servers increases"""
    val results = []
    
    for k in range(1, max_k + 1):
        val model_k = gallery_mmk(k)
        val solver_k = MVA(model_k)
        val avg_table_k = solver_k.avgTable
        
        val util = float(avg_table_k.iloc[1, 1])
        val resp_time = float(avg_table_k.iloc[1, 2])
        val queue_length = float(avg_table_k.iloc[1, 3])
// Theoretical utilization
        val rho_theory = 2.0 / k  # λ/(k×μ) = 2/(k×1)
        
        results.append((k, rho_theory, util, resp_time, queue_length))
    
    return results

val scaling_results = analyze_mmk_scaling(6)

println("k | ρ_theory | Utilization | Response Time | Queue Length | Servers Cost")
println("-" * 75)

for k, rho_theory, util, resp_time, queue_length, in scaling_results:
    val servers_cost = k  # Linear cost assumption
    println(f"{k} |   {rho_theory:.3f}   |    {util:.4f}    |     {resp_time:.4f}    |    {queue_length:.4f}     |      {servers_cost}")

println(f"\nKey Insights:")
println(f"1. Utilization decreases as 1/k (diminishing returns)")
println(f"2. Response time improves significantly with additional servers")
println(f"3. Queue length decreases dramatically with more servers")
println(f"4. Cost-benefit tradeoff: server cost vs. performance improvement")

In [ ]:
// Analyze stability boundary
println("\n=== Stability Analysis ===")

fun analyze_stability_boundary(k=3): Network {
    """Analyze system behavior near stability boundary"""
// For stability: λ < k×μ, so with μ=1, we need λ < k
    val lambda_values = np.arange(0.5, k-0.1, 0.5)
    
    println(f"M/M/{k} stability analysis (μ=1 per server, capacity={k}):")
    println(f"Arrival Rate | Utilization | Response Time | Queue Length | Status")
    println("-" * 70)
    
    for lam in lambda_values:
        try:
            val model_stab = Network(f"M/M/{k}-λ{lam}")
            val source = Source(model_stab, "Source")
            val queue = Queue(model_stab, "Queue", SchedStrategy.FCFS)
            queue.setNumberOfServers(k)
            val sink = Sink(model_stab, "Sink")
            
            val oclass = OpenClass(model_stab, "Class")
            source.setArrival(oclass, Exp(lam))
            queue.setService(oclass, Exp(1))
            
            val P = model_stab.initRoutingMatrix()
            P.addRoute(oclass, source, queue, 1.0)
            P.addRoute(oclass, queue, sink, 1.0)
            model_stab.link(P)
            
            val solver_stab = MVA(model_stab)
            val avg_table_stab = solver_stab.avgTable
            
            val util = float(avg_table_stab.iloc[1, 1])
            val resp_time = float(avg_table_stab.iloc[1, 2])
            val queue_length = float(avg_table_stab.iloc[1, 3])
            
            val rho = lam / k
            val status = "Stable" if rho < 1 else "Unstable"
            
            println(f"    {lam:4.1f}     |    {util:.4f}    |     {resp_time:.4f}    |    {queue_length:.4f}     | {status}")
            
        except Exception as e:
            println(f"    {lam:4.1f}     |    Error    |     Error     |    Error      | Unstable")

analyze_stability_boundary(3)

println(f"\nStability Condition: λ < k×μ")
println(f"For k=3, μ=1: λ must be < 3 for stability")
println(f"As λ approaches k×μ, response time and queue length grow rapidly.")